# Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Data Preprocessing

### Loading Dataset

In [ ]:
ratings = pd.read_csv('../data/rating.csv.zip')
animes = pd.read_csv('../data/anime.csv')

In [ ]:
#Cleaning ratings to only allow values greater than -1
#Cleaning animes to only allow TV Shows
ratings = ratings[ratings.rating > -1]
animes = animes[animes.type == 'TV']
ratings = ratings[ratings.anime_id.isin(animes.anime_id)]

In [ ]:
print animes.shape
print ratings.shape

### Transform Dataset

In [ ]:
#Getting the amount of differents users
nb_users = ratings.groupby("user_id").user_id.nunique().shape[0]
#Getting the amount of anime shows
nb_shows = animes.shape[0]
print nb_users
print nb_shows

In [ ]:
#Rewritting ratings dataframe to ajust user_id in sequences
#We have 69600 users in ratings but we have 73516 different users id's
#We need to create a test set, but we'll have data loss if we don't have the user_id in sequences
actual_index = ratings.iloc[0,].user_id
new_index = 0
new_list_user_id = []
for i in range(ratings.shape[0]):
    if i % 120000 == 0:
        print"{0} %".format((i / float(ratings.shape[0])) * 100.)
    if ratings.iloc[i,].user_id != actual_index:
        new_index += 1
        actual_index = ratings.iloc[i,].user_id    
    new_list_user_id.append(new_index)
print actual_index
print new_index        

In [ ]:
#Doing the same structure for anime data
mapped_anime_ids = {}
for i in range(animes.shape[0]):
    mapped_anime_ids[animes.iloc[i,].anime_id] = i

In [ ]:
ratings.head()

In [ ]:
for i in range(ratings.shape[0]):
    if i % 120000 == 0:
        print"{0} %".format((i / float(ratings.shape[0])) * 100.)
    ratings.iloc[i,].anime_id = mapped_anime_ids[ratings.iloc[i,].anime_id]

In [ ]:
ratings.tail()

In [ ]:
ratings['user_id'] = new_list_user_id

In [ ]:
ratings.tail()

In [ ]:
ratings.head()

### Splitting Data Set

In [ ]:
#Exporting new data set.
ratings.to_csv('ratings_corrected.csv')

### Working with corrected data

In [ ]:
ratings_corrected = pd.read_csv('./ratings_corrected.csv')

#### Creating Training and Test Set with Numpy Arrays

In [ ]:
training_set, test_set = train_test_split(ratings_corrected, test_size = 0.2, random_state=42)

In [ ]:
test_set.head()

In [ ]:
def convert_set(data, n_rows, n_cols):
        process_set = []
        for i  in range(n_rows):
            rated = data[data.user_id == i]
            animes_rated = rated.anime_id
            ratings_obtained = rated.rating
            ratings = np.zeros(n_cols)
            ratings[animes_rated] = ratings_obtained

            process_set.append(list(ratings))
        return process_set            

In [ ]:
training_set = convert_set(training_set, nb_users, nb_shows)

In [ ]:
test_set = convert_set(test_set, nb_users, nb_shows)

# Creating Stacker AutoEncoder Model

### Creating SAE class

In [ ]:
from types import *
import sys
class SAE(nn.Module):
    def __init__(self, input_output_size, encoder_input=20, decoder_input=20):
        super(SAE, self).__init__()
        self.encoder = nn.Linear(input_output_size, encoder_input)
        self.hidden_layers = []
        self.decoder = nn.Linear(decoder_input, input_output_size)
        self.activation = nn.Sigmoid()
        self.last_out = encoder_input
        
    def forward(self, x):
        #First step
        x = self.activation(self.encoder(x))
        #We only handle Linear and Dropout layers
        for layer in self.hidden_layers:
            if "Linear" in str(type(layer)):
                #It's a linear layer
                x = self.activation(layer(x))
            else:
                #It's a dropout layer
                x = layer(x)
        #Final Step
        x = self.decoder(x)
        return x
    
    def add_hiden_layer(self, out_features):
        new_layer = nn.Linear(self.last_out, out_features)
        self.last_out = out_features
        self.hidden_layers.append(new_layer)
        
    def add_dropout(self, p=0.5):
        new_dropout = nn.Dropout(p)
        self.hidden_layers.append(new_dropout)
        
    def print_progress(self, message):
        sys.stdout.write("\r" + message)
        sys.stdout.flush()
        
    def compile(optimizer='rmsprop', criterion='mse'):
        if type(criterion) is StringType:
            if criterion == 'mse':
                self.criterion = nn.MSELoss()
            else:
                self.criterion = nn.L1Loss()
        else:
            self.optimizer = optimizer
            if type(optimizer) is StringType:
                pass
            else:
                self.optimizer = optimizer
            
    def train(self, X, nb_epoch):
        for epoch in range(nb_epoch):
            train_loss = 0
            s = 0.
            rows, columns = X.size()
            for index in range(int(row)):
                input = Variable(X[index]).unsqueeze(0)
                target = input.clone()
                if torch.sum(target.data > 0) > 0:
                    output = self.forward(input)
                    target.require_grad = False
                    output[target == 0] = 0
                    loss = self.criterion(output, target)
                    mean_corrector = columns/float(torch.sum(target.data > 0) + 1e-10)
                    loss.backward()
                    train_loss += np.sqrt(loss.data[0] * mean_corrector)
                    s += 1.
                    optimizer.step()
                self.print_progress("epoch: {0}, training: {1}/{2}".format(epoch + 1, index + 1, rows))
            self.print_progress('epoch: {0}, training loss{1}'.format(epoch + 1, train_loss / s))
            sys.stdout.write("\n" + message)
            sys.stdout.flush()
            

In [ ]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [ ]:
training_set = torch.FloatTensor(training_set)

In [ ]:
nb_epoch = 200
for epoch in range(nb_epoch):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_shows/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


In [1]:
print "Hello World"

Hello World
